# ROM Video Analyzing
* To run this note, you need to install Openpose following [Steps to Install Openpose in ComputeCanada Cluster](https://docs.google.com/document/d/1vQCQD2iet-K1ZAjTns2aEp-q7R-w8fCQ55jVzriWLV0/edit?usp=sharing)
* To set up Jupyter Notebook and submit jobs in Compute Canada Clusters, please refer to [Steps to Use Openpose Analyzing Video](https://docs.google.com/document/d/1tozpc-KpAHVjQOx5XkW0pDYbd8IKKCD45Omk-RvFNcg/edit?usp=sharing)

In [2]:
import os
os.system('module load nixpkgs/16.09  gcc/5.4.0  cuda/8.0.44  cudnn/7.0 opencv/3.3.0  boost/1.65.1 openblas/0.2.20 hdf5/1.8.18 leveldb/1.18 mkl-dnn/0.14 python/3.5.2')
os.system('cd ~')
os.system('source openposeEnv_Python3/bin/activate')
os.system('export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/openpose_python_lib/lib:$HOME/openpose_python_lib/python/openpose:$HOME/caffe/build/lib:/cvmfs/soft.computecanada.ca/easybuild/software/2017/avx2/Compiler/gcc5.4/boost/1.65.1/lib')

0

In [3]:
# From Python
# It requires OpenCV installed for Python
import sys
import cv2
import os
from sys import platform
import argparse

import pdb
from IPython.core.debugger import Tracer

# Remember to add your installation path here
# Option b
# If you run `make install` (default path is `/usr/local/python` for Ubuntu), you can also access the OpenPose/python module from there. This will install OpenPose and the python library at your desired installation path. Ensure that this is in your python path in order to use it.
sys.path.insert(0,r'/home/lingheng/openpose_python_lib/python/openpose') 

# Parameters for OpenPose. Take a look at C++ OpenPose example for meaning of components. Ensure all below are filled
try:
    from openpose import *
except:
    raise Exception('Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?')
params = dict()
params["logging_level"] = 3
params["output_resolution"] = "-1x-1"
params["net_resolution"] = "-1x368" # if crop video, this should be changged and must be mutplies of 16.
params["model_pose"] = "BODY_25"
params["alpha_pose"] = 0.6
params["scale_gap"] = 0.3
params["scale_number"] = 1
params["render_threshold"] = 0.05
# If GPU version is built, and multiple GPUs are available, set the ID here
params["num_gpu_start"] = 0
params["disable_blending"] = False
# Ensure you point to the correct path where models are located
params["default_model_folder"] = "/home/lingheng/openpose/models/"
# Construct OpenPose object allocates GPU memory
openpose = OpenPose(params)


In [4]:
# construct the argument parser and parse the arguments
args = dict()
args['video']='/home/lingheng/project/lingheng/ROM_Video_Process/Openpose_Video_Analysis_Code/test_video.mp4'
args['output_directory']='/home/lingheng/project/lingheng/ROM_Video_Process/Openpose_Video_Analysis_Code'

if args.get("video", None) is None:
    raise Error("No input video!!")
# otherwise, we are reading from a video file
else:
    camera = cv2.VideoCapture(args["video"])
# frames per second (fps) in the raw video
fps = camera.get(cv2.CAP_PROP_FPS)
frame_count = 1
print("Raw frames per second: {0}".format(fps))
# prepare to save video
(grabbed, frame) = camera.read()
## downsample frame
#downsample_rate = 0.5
#frame = cv2.resize(frame,None,fx=downsample_rate, fy=downsample_rate, interpolation = cv2.INTER_LINEAR)
# crop frame
original_h, original_w, channels= frame.shape
top_edge = int(original_h*(1/10))
down_edge = int(original_h*1)
left_edge = int(original_w*(1/5))
right_edge = int(original_w*(4/5))
frame_cropped = frame[top_edge:down_edge,left_edge:right_edge,:].copy() # must use copy(), otherwise slice only return address i.e. not hard copy

cropped_h, cropped_w, channels = frame_cropped.shape
fwidth = cropped_w 
fheight = cropped_h
print("Frame width:{}, Frame height:{}.".format(cropped_w , cropped_h))
# Define the polygon of Core Interest Area
point_1 = [int(0.17 * cropped_w), int(0.20 * cropped_h)]
point_2 = [int(0.17 * cropped_w), int(0.62 * cropped_h)]
point_3 = [int(0.44 * cropped_w), int(0.82 * cropped_h)]
point_4 = [int(0.61 * cropped_w), int(0.72 * cropped_h)]
point_5 = [int(0.61 * cropped_w), int(0.20 * cropped_h)]
core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])

# get output video file name
file_path = args["video"].split('/')
file_name, _= file_path[-1].split('.')
fourcc = cv2.VideoWriter_fourcc(*'XVID')

out_camera_frame_whole = cv2.VideoWriter(os.path.join(args['output_directory'],'{}_processed_whole.avi'.format(file_name)),fourcc, fps, (fwidth,fheight))
out_camera_frame_core = cv2.VideoWriter(os.path.join(args['output_directory'],'{}_processed_core.avi'.format(file_name)),fourcc, fps, (fwidth,fheight))
out_camera_frame_margin = cv2.VideoWriter(os.path.join(args['output_directory'],'{}_processed_margin.avi'.format(file_name)),fourcc, fps, (fwidth,fheight))
# get output estimated occupancy file name
out_occupancy_whole = os.path.join(args['output_directory'],'{}_processed_occupancy_whole.txt'.format(file_name))
out_occupancy_core = os.path.join(args['output_directory'],'{}_processed_occupancy_core.txt'.format(file_name))
out_occupancy_margin = os.path.join(args['output_directory'],'{}_processed_occupancy_margin.txt'.format(file_name))


Raw frames per second: 30.00063026534171
Frame width:1152, Frame height:972.


In [5]:
camera.get(cv2.CAP_PROP_FRAME_COUNT)

1428.0

In [ ]:
# loop over the frames of the video
total_frame_number = camera.get(cv2.CAP_PROP_FRAME_COUNT)
for frame_count in range(int(total_frame_number)):
    if frame_count % 200 == 0:
        print('Processing frame: {}'.format(frame_count))
    (grabbed, frame) = camera.read()
    if grabbed == True:
        time = camera.get(cv2.CAP_PROP_POS_MSEC) #Current position of the video file in milliseconds.
        ## downsample frame
        #frame = cv2.resize(frame,None,fx=downsample_rate, fy=downsample_rate, interpolation = cv2.INTER_LINEAR)
        # crop frame
        frame_cropped = frame[top_edge:down_edge,left_edge:right_edge,:].copy() # must use copy()
        
        # 1. Whole Interest Area
        # Output keypoints and the image with the human skeleton blended on it
        #    (num_people, 25_keypoints, x_y_confidence) = keypoints_whole_interest_area.shape
        keypoints_whole_interest_area, output_image_whole_interest_area = openpose.forward(frame_cropped, True)

        # 2. Core Interest Area
        core_interest_area_mask = np.zeros(frame_cropped.shape[:2], np.uint8)
        cv2.drawContours(core_interest_area_mask, [core_interest_area_polygon], -1, (255, 255, 255), -1, cv2.LINE_AA)
        core_interest_area = cv2.bitwise_and(frame_cropped, frame_cropped, mask=core_interest_area_mask)
        keypoints_core_interest_area, output_image_core_interest_area = openpose.forward(core_interest_area, True)
        # 3. Margin Interest Area
        margin_interest_area = cv2.bitwise_xor(frame_cropped, core_interest_area)
        keypoints_margin_interest_area, output_image_margin_interest_area = openpose.forward(margin_interest_area, True)
        
        # draw the text and timestamp on the frame
        occupancy_whole = keypoints_whole_interest_area.shape[0]
        occupancy_core = keypoints_core_interest_area.shape[0]
        occupancy_margin = keypoints_margin_interest_area.shape[0]
        
        cv2.putText(output_image_whole_interest_area, "Whole Occupancy: {}".format(occupancy_whole), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(output_image_core_interest_area, "Core Occupancy: {}".format(occupancy_core), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(output_image_margin_interest_area, "Margin Occupancy: {}".format(occupancy_margin), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # save estimated occupancy data
        with open(out_occupancy_whole, "a+") as fh:
           fh.write("{},{}\r".format(time, occupancy_whole))
        with open(out_occupancy_core, "a+") as fh:
           fh.write("{},{}\r".format(time, occupancy_core))
        with open(out_occupancy_margin, "a+") as fh:
           fh.write("{},{}\r".format(time, occupancy_margin))
        # save processed videos
        out_camera_frame_whole.write(output_image_whole_interest_area)
        out_camera_frame_core.write(output_image_core_interest_area)
        out_camera_frame_margin.write(output_image_margin_interest_area)
    else:
        # Pass this frame if cannot grab an image.
        print('Frame: {}, grabbed={} and frame={}'.format(frame_count, grabbed, frame))

Processing frame: 0
> <ipython-input-6-a1a75e2f5fe8>(20)<module>()
-> core_interest_area_mask = np.zeros(frame_cropped.shape[:2], np.uint8)
(Pdb) keypoints_whole_interest_area
array([[[2.4658525e+02, 4.1497391e+02, 4.4426811e-01],
        [2.2261572e+02, 4.3911035e+02, 9.5780784e-01],
        [2.4389468e+02, 4.3923114e+02, 8.9719373e-01],
        [2.4664441e+02, 4.7398798e+02, 8.0316323e-01],
        [2.4657712e+02, 4.9805737e+02, 7.0036328e-01],
        [2.0382619e+02, 4.3656747e+02, 9.3514740e-01],
        [2.0100603e+02, 4.6062299e+02, 7.7358425e-01],
        [2.0644493e+02, 4.8193091e+02, 4.3585134e-01],
        [2.2238084e+02, 4.9531775e+02, 8.8509297e-01],
        [2.2782591e+02, 4.9794751e+02, 8.5150599e-01],
        [2.2528212e+02, 5.3813483e+02, 8.8107777e-01],
        [2.2527803e+02, 5.7564294e+02, 8.2000774e-01],
        [2.0650104e+02, 4.9532825e+02, 8.4267753e-01],
        [2.0394464e+02, 5.3282416e+02, 8.5341430e-01],
        [2.0371606e+02, 5.7299536e+02, 8.8641548e-01],

In [ ]:
keypoints_whole_interest_area.shape